In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
import random

random.seed(0)

Sobre la data:

Desde VSE++ (https://github.com/fartashf/vsepp)

Particiones F30K: https://www.kaggle.com/datasets/shtvkumar/karpathy-splits

Data: https://www.kaggle.com/datasets/hsankesara/flickr-image-dataset

Extraccion con EfNetV2 L 30K (13 min) 
https://www.kaggle.com/code/mariocarlosmallea/extrac-feautures-f30k/edit/run/108300793

Extraccion MPNET 3 (min): 

Matriz en orden de aparición

## Construction for MSCOCO

In [36]:
#parche faltaba 1
path= 'COCO/aux_/' 

visual_matrix1= np.load(path+"/EFFV2L_COCO_train1.npy")
visual_matrix2= np.load(path+"/EFFV2L_COCO_train2.npy")
visual_matrix_= np.load(path+"/EFFV2L_COCO_train2_.npy")

v_m= np.concatenate([visual_matrix1,visual_matrix_],axis=0)
v_m= np.concatenate([v_m,visual_matrix2],axis=0)
np.save("EFFV2L_COCO.npy",v_m)

In [4]:
path= 'COCO/annotations/' 
dataset_train = jsonmod.load(open(path+"captions_train2014.json", 'r'))
dataset_val = jsonmod.load(open(path+"captions_val2014.json", 'r'))

#index of captions
train_id=np.load(path+"coco_train_ids.npy")
val_id=np.load(path+"coco_dev_ids.npy")[:5000]
test_id=np.load(path+"coco_test_ids.npy")

In [5]:
val_id.shape

(5000,)

SI ESTAN EN ORDEN DEBO CONSULTAR LAS IMG ASI

In [108]:
for i in test_id[:5]:
    for c in dataset_val['annotations']:
        if c["id"]==i:
            print(c)
            break

{'image_id': 391895, 'id': 770337, 'caption': 'A man with a red helmet on a small moped on a dirt road. '}
{'image_id': 391895, 'id': 771687, 'caption': 'Man riding a motor bike on a dirt road on the countryside.'}
{'image_id': 391895, 'id': 772707, 'caption': 'A man riding on the back of a motorcycle.'}
{'image_id': 391895, 'id': 776154, 'caption': 'A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud-wreathed mountains. '}
{'image_id': 391895, 'id': 781998, 'caption': 'A man in a red shirt and a red hat is on a motorcycle on a hill side.'}


In [32]:
path= 'COCO/annotations/' 
dataset_train = jsonmod.load(open(path+"captions_train2014.json", 'r'))
dataset_val = jsonmod.load(open(path+"captions_val2014.json", 'r'))

#index of captions
train_id=np.load(path+"coco_train_ids.npy")
val_id=np.load(path+"coco_dev_ids.npy") #[:5000]
test_id=np.load(path+"coco_test_ids.npy")

dataset= []
map_img=[] #index like map
map_text=[]


for i in tqdm(train_id):
    if i not in map_text:
        map_text.append(i)
    for c in dataset_train['annotations']:
        if c["id"]==i:
           if c["image_id"] not in map_img:
                map_img.append(c["image_id"])
                dict_={}
                dict_["split"]="train"
                dict_["sentids"]=[map_text.index(i)]
                dict_["imgid"]= map_img.index(c["image_id"]) #c["image_id"]
                dataset.append(dict_)
           else:
                dataset[map_img.index(c["image_id"])]["sentids"].append(map_text.index(i))


for i in tqdm(val_id):
    if i not in map_text:
        map_text.append(i)
    for c in dataset_val['annotations']:
        if c["id"]==i:
           if c["image_id"] not in map_img:
                map_img.append(c["image_id"])
                dict_={}
                dict_["split"]="val"
                dict_["sentids"]=[map_text.index(i)]
                dict_["imgid"]=map_img.index(c["image_id"])
                dataset.append(dict_)
           else:
                dataset[map_img.index(c["image_id"])]["sentids"].append(map_text.index(i))


for i in tqdm(test_id):
    if i not in map_text:
        map_text.append(i)
        
    for c in dataset_val['annotations']:
        if c["id"]==i:
           if c["image_id"] not in map_img:
                map_img.append(c["image_id"])
                dict_={}
                dict_["split"]="test"
                dict_["sentids"]=[map_text.index(i)]
                dict_["imgid"]=map_img.index(c["image_id"])
                dataset.append(dict_)
           else:
                dataset[map_img.index(c["image_id"])]["sentids"].append(map_text.index(i))
                
np.save("map_img_coco.npy",map_img)
np.save("map_text_coco.npy",map_text)
np.save("dataset_coco.npy", dataset)

100%|████████████████████████████████████████████████████████████████████████████| 25000/25000 [20:26<00:00, 20.38it/s]


In [20]:
#restval

path= 'COCO/' 
dataset_val = jsonmod.load(open(path+"annotations/captions_val2014.json", 'r'))

dataset= list(np.load(path+"dataset_coco.npy",allow_pickle=True))
map_img=list(np.load(path+"map_img_coco.npy",allow_pickle=True))
map_text=list(np.load(path+"map_text_coco.npy",allow_pickle=True))

#index of captions
restval_id=np.load(path+"annotations/coco_restval_ids.npy")


for i in tqdm(restval_id):
    if i not in map_text:
        map_text.append(i)
    for c in dataset_val['annotations']:
        if c["id"]==i:
           if c["image_id"] not in map_img:
                map_img.append(c["image_id"])
                dict_={}
                dict_["split"]="train"
                dict_["sentids"]=[map_text.index(i)]
                dict_["imgid"]= map_img.index(c["image_id"]) #c["image_id"]
                dataset.append(dict_)
           else:
                dataset[map_img.index(c["image_id"])]["sentids"].append(map_text.index(i))


np.save("map_img_coco_restval.npy",map_img)
np.save("map_text_coco_restval.npy",map_text)
np.save("dataset_coco_restval.npy", dataset)

100%|████████████████████████████████████████████████████████████████████████| 152520/152520 [2:44:48<00:00, 15.42it/s]


## Data

In [4]:
pip install tqdm

     ---------------------------------------- 0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json as jsonmod
import numpy as np
import random
import time
import pickle
from tqdm import tqdm



In [4]:
name_data= "coco" #"coco"

In [5]:

if name_data=="F30K":
    path= 'F30K' 
    dataset = jsonmod.load(open(path+"/dataset_flickr30k.json", 'r'))['images']
    
else: #coco structure to (like) F30K

    path= 'COCO' 
    map_img=np.load(path+"/map_img_coco_restval.npy",allow_pickle=True)
    map_text=np.load(path+"/map_text_coco_restval.npy",allow_pickle=True)
    dataset= np.load(path+"/dataset_coco_restval.npy",allow_pickle=True)
                    
                    
all_val_indices_img,all_test_indices_img, all_val_indices_text,all_test_indices_text=[],[],[],[]

for i in range(len(dataset)):
  if dataset[i]["split"]=="val":
    all_val_indices_img.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      all_val_indices_text.append(t)


  if dataset[i]["split"]=="test":
    all_test_indices_img.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      all_test_indices_text.append(t)

print("Train-Val-Test")  
print(len(dataset), len(all_val_indices_img), len(all_test_indices_img))

                

Train-Val-Test
123287 5000 5000


dataset -> [ ... (imgid, [5 textos] )...]

In [6]:

dataset_= []
random_train=[]
for i in range(len(dataset)):
  if dataset[i]["split"]=="test":
    dataset_.append(dataset[i])
    
  if dataset[i]["split"]=="train" :
    dataset_.append(dataset[i])
    
dataset= list(dataset_)


In [21]:
dataset_= []
#random_train=[]
for i in range(len(dataset)):
  if dataset[i]["split"]=="val":
    dataset_.append(dataset[i])
    
  #if dataset[i]["split"]=="train" :
    #random_train.append(dataset[i])

In [24]:
random_train_half=list(np.random.choice(random_train,int(100) , replace=False))

In [2]:
#random_train_half=list(np.random.choice(random_train,int( (31014-1000-1014)/2) , replace=False))
#random_train_half=list(np.random.choice(random_train,int( (123287-5000-5000)/2) , replace=False))

In [4]:
#np.save("random_val_halfTrain_F30K.npy",np.array(random_train_half))
#np.save("random_val_halfTrain_coco.npy",np.array(random_train_half))

In [22]:
#random_train_half= np.load("random_val_halfTrain_F30K.npy", allow_pickle=True)
random_train_half= np.load(path+"/random_val_halfTrain_coco.npy", allow_pickle=True)

In [23]:
dataset= list(dataset_)+list(random_train_half)

In [6]:
dataset=list(np.random.choice(dataset[:10000],1000, replace=False)) + list(np.random.choice(dataset[10000:],1000,replace=False))
#print(len(dataset))
#print(len(dataset[0]))

In [ ]:
#dataset[0].keys()

In [ ]:
#dataset[0]["sentids"]

In [ ]:
#dataset[0]["imgid"]

In [ ]:
#dataset[0]["split"]

In [7]:
train_indices=[]
data_text=[] #querys train

 #dictionary ground truth retrieval img o text
all_r_img, all_r_text= {},{} #train+val+test
val_r_img, val_r_text, test_r_img, test_r_text= {},{},{},{} 

for i in range(len(dataset)):
  if dataset[i]["split"]=="train":
    train_indices.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      data_text.append(t)
      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]


  if dataset[i]["split"]=="val":
    
    for t in dataset[i]["sentids"]:
      val_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      val_r_img[t]= dataset[i]["imgid"]
        
      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]

  if dataset[i]["split"]=="test":
    
    for t in dataset[i]["sentids"]:
      test_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      test_r_img[t]= dataset[i]["imgid"]
        
      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]


print("Train-Val-Test")
print("img: ", len(train_indices),len(val_r_text.keys()), len(test_r_text.keys()))
print("text: ",len(data_text), len(val_r_img.keys()), len(test_r_img.keys()))
print("-"*20)
# captions order for modeling e.g: [id 999] ->[position 0] with .index
universe_text= data_text+ list(val_r_img.keys())+list(test_r_img.keys()) 
#img order for modeling
universe_img= train_indices+list(val_r_text.keys())+ list(test_r_text.keys())
print("All Img-Text")
print(len(universe_img),len(universe_text))


Train-Val-Test
img:  113287 0 5000
text:  566435 0 25000
--------------------
All Img-Text
118287 591435


In [8]:
import sklearn.preprocessing
if name_data=="F30K":
    
    textual_matrix= np.load(path+"/MPNET_F30K.npy")
    visual_matrix= np.load(path+"/EFFV2L_F30K.npy")
    
else: #coco    
    textual_matrix= np.load(path+"/MPNET_COCO.npy")
    visual_matrix= np.load(path+"/EFFV2L_COCO.npy")
    
print(textual_matrix.shape, visual_matrix.shape)


visual_matrix=sklearn.preprocessing.normalize(visual_matrix[universe_img,:], norm='l2', axis=0) #visual_matrix[universe_img,:]/np.max(visual_matrix)
textual_matrix=textual_matrix[universe_text,:]#/np.max(textual_matrix)
print(textual_matrix.shape, visual_matrix.shape)



(616435, 768) (123287, 1280)
(591435, 768) (118287, 1280)


In [9]:
m_validation=[0.2,0.4]  
f_validation= [768,896,1024] # VSE++=1024
m_2_validation= [1e-3,1e-2, 0.2,0.4,0.6]

## caption description

In [61]:
pip install sentence-transformers

     ---------------------------------------- 86.0/86.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 6.3/6.3 MB 50.5 MB/s eta 0:00:00
     ------------------------------------- 162.5/162.5 MB 25.2 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 72.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 93.8 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 70.2 MB/s eta 0:00:00
     ------------------------------------- 190.3/190.3 kB 11.3 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 13.2 MB/s eta 0:00:00
     ---------------------------------------- 267.8/267.8 kB ? eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB ? eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256

In [34]:
path= 'COCO/annotations/' 
dataset_train = jsonmod.load(open(path+"captions_train2014.json", 'r'))
dataset_val = jsonmod.load(open(path+"captions_val2014.json", 'r'))

In [35]:
path= 'COCO/' 
map_text=np.load(path+"map_text_coco_restval.npy",allow_pickle=True)

In [36]:
map_text.shape

(616435,)

In [57]:
textos=[]

for c in tqdm(map_text):
    flag=True
    for i in dataset_train["annotations"]:
        if i["id"]==c:
            textos.append(i["caption"])
            flag=False
            break
    if flag:
        for i in dataset_val["annotations"]:
            if i["id"]==c:
                textos.append(i["caption"])
                break

100%|████████████████████████████████████████████████████████████████████████| 616435/616435 [8:36:53<00:00, 19.88it/s]


In [58]:
np.save("captions_coco_.npy", textos)

In [59]:
len(textos)

616435

In [ ]:
textos=[]
indices=[]
for i in dataset:
    for j in i["sentences"]:
        textos.append(j["raw"])
        indices.append( (j["imgid"],j["sentid"]) )

In [62]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [63]:
embeddings_t= model.encode(textos, show_progress_bar=True)

Batches:   0%|          | 0/19264 [00:00<?, ?it/s]

In [64]:
embeddings_t.shape

(616435, 768)

In [65]:
np.save("MPNET_COCO.npy",embeddings_t)
#np.save("MPNET_F30K.npy",embeddings_t)

In [ ]:
np.save("MPNET_F30K_indices.npy",indices)

In [ ]:
indice_mpnet= np.load(path+"/MPNET_F30K_indices.npy")

## Evaluation

In [10]:

import concurrent.futures
from collections import defaultdict
from scipy.sparse import csr_matrix
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing
import gc

def get_batch_data(data, index, size):
    """
    For minibatch training
    """
  
    column_1 = []
    column_2 = []
    for i in range(index, index + size):
        line = data[i]
        column_1.append(int(line[0]))#text
        column_2.append(int(line[1]))#img
    return np.array(column_1), np.array(column_2)


_model = None
_universe_query= None
_retrieval= None
evaluation_Q_= None
_universe_value= None
_list_rank= None
_at_k= None
evaluation_V_=None
def evaluate_model(model, nDCG, split, retrieval): #i2i
    """
    evaluate model
    """
    global _model
    global _universe_query
    global _universe_value
    global _retrieval
    global evaluation_Q_
    global _list_rank
    global _at_k
    global evaluation_V_
    
    _list_rank= []
    _retrieval=retrieval
    _model = model
    test_users = []
    results=[]
    output= []
    _at_k=10
    
    #-----------------------------------------------------------
    if _retrieval=="t2i":
      _universe_query= universe_text
      _universe_value= universe_img
      if split=="val":
        evaluation_Q_= val_r_img
        evaluation_V_= list(val_r_text.keys())
      if split=="test": #test
        evaluation_Q_= test_r_img
        evaluation_V_= list(test_r_text.keys())
    
    else:  #i2t
      _universe_query= universe_img
      _universe_value= universe_text
      if split=="val":
        evaluation_Q_= val_r_text
        evaluation_V_= list(val_r_img.keys())
      else: #test
        evaluation_Q_= test_r_text
        evaluation_V_= list(test_r_img.keys())

    evaluation_Q_num = len(evaluation_Q_.keys())
    
    #if i2i:
     #   return t2i( _model, split ,aux_mod=True)
    
    result_acumulate= np.array([0.] * 3) # 3 metrics
    for q in evaluation_Q_.keys():
        result_acumulate+=eval_one_user(q,split) 

    ret = result_acumulate / evaluation_Q_num #avg 
    ret = list(ret)

    results.append(ret)
    output.append(results)
    

    if nDCG:
        ndcg_i2t,ndcg_t2i= i2t( _model,split) ,t2i( _model, split)
        ndcg_i2i, ndcg_t2t= t2i( _model, split ,aux_mod=True), t2t( _model,split)
        output.append((ndcg_i2t,ndcg_t2i, ndcg_i2i,ndcg_t2t))
      
    
    return output



def eval_one_user(query,split):
    """
    each evaluation
    """
    
    rating = _model.predict(_universe_query.index(query), _retrieval,split)
    #gc.collect()
    if _retrieval=="t2i":
      ground_truth = [ evaluation_Q_[query] ] #one img
    else: #i2t
      ground_truth =  evaluation_Q_[query] #list
      #print(query)
    
    rank_=np.argsort(-rating)
    rank= [evaluation_V_[i]  for i in rank_[:_at_k]]


    result = []
    result.append(recall_at_k(rank, ground_truth, 1))
    result.append(recall_at_k(rank, ground_truth, 5))
    result.append(recall_at_k(rank, ground_truth, 10))

    
    return np.array(result)



def recall_at_k(rank, ground_truth, k):
    """
    Recall
    """
    #return 100 * len(set(rank[:k]) & set(ground_truth)) / len(ground_truth)
    if len(set(rank[:k]) & set(ground_truth))>=1:
        return 100
    else:
        return 0


def calculate_diversity(_list_rank):
    """ 
    max_= len(_universe_value)    
    coverage= set()
    label_binarizer = sklearn.preprocessing.LabelBinarizer()
    label_binarizer.fit(range(max_))
    list_=[]
    dict_count={}
    for rank in _list_rank: # each RANKING
        coverage= coverage | set(rank)
        list_.append( np.sum(label_binarizer.transform(rank),axis=0) )
        for i in rank:
            if i not in dict_count:
                dict_count[i]=1
            else:
                dict_count[i]+=1
    recs_content=csr_matrix(np.array(list_) )
    #calculate similarity scores for all items in list
    similarity = cosine_similarity(X=recs_content, dense_output=False)
    #get indicies for upper right triangle w/o diagonal
    upper_right = np.triu_indices(similarity.shape[0], k=1)
    diversity_cos = np.mean(similarity[upper_right])
    
    """
    dict_count={}
    for rank in _list_rank: # each RANKING
        #coverage= coverage | set(rank)
        #list_.append( np.sum(label_binarizer.transform(rank),axis=0) )
        for i in rank:
            if i not in dict_count:
                dict_count[i]=1
            else:
                dict_count[i]+=1
    #novelty-diver
    nov=0
    for rank in _list_rank: # each RANKING
        self_information=0
        for i in rank:
            self_information += np.sum( -np.log2(dict_count[i]/len(evaluation_Q_)) )
        nov+=self_information/_at_k #top
    
    return (0., 0., nov/len(evaluation_Q_)) #(diversity_cos, len(coverage)/max_, nov/len(evaluation_Q_))
       


In [11]:
# (C) Mathieu Blondel, November 2013
# License: BSD 3 clause
#Original https://github.com/mesnico/TERAN/blob/master/evaluate_utils/dcg.py

import numpy as np
import os

class DCG:
    def __init__(self, rank=25, relevance_methods=['rougeL'], split="val", data=name_data): # config, n_queries, split
        self.rank = rank
        self.relevance_methods = relevance_methods
        #n_queries=npts
        if data=="F30K":
            if split=="val":
                relevance_filenames= [path+"/f30k-val-rougeL.npy", path+"/f30k-val-spice.npy"] #5.140.980= 5070x1014
                self.npts= 1014 #images
            else:#test
                relevance_filenames= [path+"/f30k-test-rougeL.npy", path+"/f30k-test-spice.npy"] #500.000=5000x1000
                self.npts= 1000 #images
        else: #coco
            if split=="val":
                relevance_filenames= [path+"/coco-val-rougeL.npy", path+"/coco-val-spice.npy"] #(5000,1000)
                self.npts=1000 # [:5000] line 60 https://github.com/mesnico/TERN/blob/master/data.py
            else:
                relevance_filenames= [path+"/coco-test-rougeL.npy", path+"/coco-test-spice.npy"] #(25000,5000)
                self.npts=5000 #images
        
        self.relevances = [np.memmap(f, dtype=np.float32, mode='r') for f in relevance_filenames]
        
        # una fila se lleva a una matriz de n_queries filas y lo que quede de columnas (en orden)
        for r in self.relevances:
            r.shape = (self.npts*5, -1) #debe dividir a 5 millones

    def compute_ndcg(self, query_id, sorted_indexes, fold_index=0, retrieval='t2i'):
        """
        sorted_indexe: all sorted indices
        query_id: image retrieval \in [0,5000], sentence retrieval \in [0,1000]
        # npts: images.shape[0] // 5
        fold_index= for 5fold cross-validation, only for MSCOCO
        """
        sorted_indexes = sorted_indexes[:self.rank] #TOP@K
        if retrieval == 't2i':
            query_base = self.npts * 5 * fold_index #0
            #caption query pertenece a  [0,5000] valores de relevancia en vector [0,1000] imagenes
            # (5000,1000)
            relevances = [r[query_base + query_id, fold_index * self.npts : (fold_index + 1) * self.npts] for r in self.relevances]
        elif retrieval == 'i2t':
            query_base = self.npts * fold_index #0
            #  valores de relevancia en vector [0,5000] captions, image query pertenece a  [0,1000] 
            relevances = [r[fold_index * self.npts * 5 : (fold_index + 1) * self.npts * 5, query_base + query_id] for r in self.relevances]
        # por métrica: ndcg(valor de relevancia, posición del ranking) 
        #print(r.shape)
        ndcg_scores = [ ndcg_from_ranking(r, sorted_indexes)  for r in relevances]
        out = {k: v for k, v in zip(self.relevance_methods, ndcg_scores)}
        return out


    


def ndcg_from_ranking(y_true, ranking):

    # El orden del muestreo es irrelevante en ndcg_from_ranking o dcg_score:
    # [0.5, 0.3, 0.2], [2, 1, 0] == [0.2, 0.3, 0.5], [0, 1, 2]
    # lo que importan es el mach (relevancia,posición del ranking)
    
    """Normalized discounted cumulative gain (NDCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    NDCG @k : float
    """
    k = len(ranking)
    best_ranking = np.argsort(y_true)[::-1]#indices de mayor a menor valores
    best = dcg_from_ranking(y_true, best_ranking[:k]) #el mejor posible 
    if best == 0:
        return 0
    return dcg_from_ranking(y_true, ranking) / best

def dcg_from_ranking(y_true, ranking):
    #Por ejemplo en el caso binario:
    #dcg_from_ranking([1, 0, 0], [2, 1, 0])=0.5= 0+0+1/log_2(4)
    #dcg_from_ranking([1, 0, 0], [0, 1, 2])=1.0= 1/log_2(2)+0+0
    #Además, corta solo para el top@K
    """Discounted cumulative gain (DCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels). #comunmente binario
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    DCG @k : float
    """
    y_true = np.asarray(y_true)
    ranking = np.asarray(ranking)
    #ordena y toma el subconjunto de los valores de y_true con respecto al ranking asociado
    rel = y_true[ranking] #normalmente esta es binaria
    gains = 2 ** rel - 1 #en el caso continuo, [0,1] y no {0,1}
    discounts = np.log2(np.arange(len(ranking)) + 2)
    return np.sum(gains / discounts)#div pointwise


#Original https://github.com/mesnico/TERAN/blob/master/evaluation.py


#n_queries= 5000 #img son 1000 files son 5000000==5000*1000


def i2t( model, split, n_ndcg=True, d_n=True, top=25):
    """
    TEXT RETRIEVAL
    
    model: Dado un img query el modelo debe dar el ranking asociado (largo 5000, de 0 a 4999) en orden de aparición!!!
    splt: "val" or "test"
    n_ndcg: novelty ndcg
    d_n: other list metrics diversity and novelty of lists
    """
    output=[]
    
    if split=="val":
      eval_indices= list(val_r_text.keys()) #img
      all_indices= all_val_indices_img #original order  
      eval_values= list(val_r_img.keys()) #text
      all_values= all_val_indices_text #original order
    else:#test
      eval_indices= list(test_r_text.keys()) #img
      all_indices= all_test_indices_img #original order 
      eval_values= list(test_r_img.keys())#text
      all_values= all_test_indices_text #original order

    

    ndcg_scorer = DCG(rank=top, relevance_methods=['rougeL', 'spice'], split=split)
    rougel_ndcgs = np.zeros(ndcg_scorer.npts)
    spice_ndcgs = np.zeros(ndcg_scorer.npts)
    div_rankings=[]
    
    nov_rankings= []
    
    
    for index in range(len(eval_indices)): #range(npts): #each QUERY
        rating= model.predict( universe_img.index(eval_indices[index]) , "i2t", split)      
        if d_n:
            rank_=np.argsort(-rating)
            div_rankings.append( rank_[:top] )
        # positions in order wrt rating (only for evaluation)    
        indices_= np.argsort( -np.array( rating) ) 
        # get original position (in order by the model)
        inds= [all_values.index(eval_values[i]) for i in indices_] #order of values = eval_values
        
        rougel_ndcgs[index], spice_ndcgs[index] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[index]), inds,retrieval='i2t').values()
        
        if n_ndcg:
            nov_rankings.append(inds)
    
    
    mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
    mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
    output.append( (mean_rougel_ndcg, mean_spice_ndcg) )
    
    if n_ndcg:
        eval_indices_all=[all_indices.index(i) for i in eval_indices]
        rankings={k: v for k, v in zip(eval_indices_all,nov_rankings)}
        
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="rougeL")
        novedad_rougeL=myAlpha.calculate()
        
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="spice")
        novedad_spice=myAlpha.calculate()
        
        output.append((novedad_rougeL,novedad_spice))
    if d_n:
        cos_div,coverage,nov= calculate_diversity(div_rankings)
        output.append((coverage,cos_div,nov))
    return output


def t2i( model,split,n_ndcg=True, d_n=True, aux_mod=False,top=25):
    #aux_mod for i2i
    output=[]
    if split=="val":
      eval_indices= list(val_r_img.keys()) #text
      all_indices= all_val_indices_text 
      eval_values= len(val_r_text.keys()) #img
      all_values= all_val_indices_img 
    else:
      eval_indices= list(test_r_img.keys())
      all_indices= all_test_indices_text  
      eval_values= list(test_r_text.keys())
      all_values= all_test_indices_img 


    ndcg_scorer = DCG( rank=top, relevance_methods=['rougeL', 'spice'],split=split)
    rougel_ndcgs = np.zeros(5*ndcg_scorer.npts)
    spice_ndcgs = np.zeros(5*ndcg_scorer.npts)
    
    nov_rankings= []
    div_rankings=[]
    
    
    for index in range( len(eval_indices)//5 ): 
        
        for i in range(5): #5 query captions
            if aux_mod==False: #t2i
                rating= model.predict( universe_text.index(eval_indices[5 * index + i]), "t2i", split)
            #i2i
            else:#t->i-> retrieval: each t have a diferent rel wrt each grup of images.
                rating= model.predict( universe_img.index(eval_values[index]) , "i2i",split)
            
            if d_n:
                rank_=np.argsort(-rating)
                div_rankings.append( rank_[:top] )
            
            indices_= np.argsort( -np.array(rating) ) 
            inds= [all_values.index(eval_values[j]) for j in indices_] 

            rougel_ndcgs[5 * index + i], spice_ndcgs[5 * index + i] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[5 * index + i]), inds, retrieval='t2i').values()
            if n_ndcg:
                nov_rankings.append(inds)
            
            
            
    mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
    mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
    output.append((mean_rougel_ndcg, mean_spice_ndcg))
    
    if n_ndcg:
        eval_indices_all=[all_indices.index(i) for i in eval_indices]
        rankings={k: v for k, v in zip( eval_indices_all, nov_rankings)}
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "t2i",ndcg_scorer=ndcg_scorer, method="rougeL")
        novedad_rougeL=myAlpha.calculate()
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "t2i",ndcg_scorer=ndcg_scorer, method="spice")
        novedad_spice=myAlpha.calculate()
        output.append((novedad_rougeL,novedad_spice))
        
    if d_n:
        cos_div,coverage,nov= calculate_diversity(div_rankings)
        output.append((coverage,cos_div,nov))
    
    return output


def t2t( model, split,n_ndcg=True, d_n=True,top=25):

    output=[]
   
    if split=="val":
      #eval_indices= list(val_r_text.keys()) #img
      all_indices= all_val_indices_img #img 
      all= all_val_indices_text 
      eval_dic= val_r_img
    else:
      #eval_indices= list(test_r_text.keys()) #img
      all_indices= all_test_indices_img #img 
      all= all_test_indices_text
      eval_dic= test_r_img
      
    eval_values= list(eval_dic.keys()) # texts
    

    ndcg_scorer = DCG( rank=top, relevance_methods=['rougeL', 'spice'], split=split)
    rougel_ndcgs = np.zeros(5*ndcg_scorer.npts)
    spice_ndcgs = np.zeros(5*ndcg_scorer.npts)
    querys=[]
    
    nov_rankings= []
    div_rankings=[]
    for index in range( len(eval_values)): 
        rating= model.predict( universe_text.index(eval_values[index]) , "t2t",split)
        if d_n:
            rank_=np.argsort(-rating)
            div_rankings.append( rank_[:top] )
        
        
        indices_= np.argsort( -np.array(rating) ) 
        inds= [all.index(eval_values[i]) for i in indices_] 
        #t->i->index
        img_query_from_text=all_indices.index(eval_dic[eval_values[index]]) 
        querys.append(img_query_from_text)
        
        rougel_ndcgs[index], spice_ndcgs[index] = ndcg_scorer.compute_ndcg( img_query_from_text, inds,retrieval='i2t').values()
        if n_ndcg:
            nov_rankings.append(inds)
    
        
    mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
    mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
    output.append((mean_rougel_ndcg, mean_spice_ndcg))
    
    if n_ndcg:
        rankings={k: v for k, v in zip(querys,nov_rankings)}
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="rougeL")
        novedad_rougeL=myAlpha.calculate()
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="spice")
        novedad_spice=myAlpha.calculate()
        output.append((novedad_rougeL,novedad_spice))
        
    if d_n:
        cos_div,coverage,nov= calculate_diversity(div_rankings)
        output.append((coverage,cos_div,nov))
    
    return output



In [12]:
class AlphaNDCG(object):
 def __init__(self, alpha=0.5, top=25,rankings=None,  retrieval= "i2t",ndcg_scorer=None, method= None):
  self.alpha = alpha
  self.fold_index=0
  self.retrieval= retrieval
  self.all_rankings= rankings
  self.ndcg_scorer= ndcg_scorer
  self.mode= ['rougeL', 'spice'].index(method)
  self.top= top
  self.npts= self.ndcg_scorer.npts


 def relevance(self, query_id):
    

    if self.retrieval == 't2i':
                query_base = self.npts * 5 * self.fold_index #0
                relevances = [r[query_base + query_id, self.fold_index * self.npts : (self.fold_index + 1) * self.npts] for r in self.ndcg_scorer.relevances]
    if self.retrieval == 'i2t':
                query_base = self.npts * self.fold_index #0
                relevances = [r[self.fold_index * self.npts * 5 : (self.fold_index + 1) * self.npts * 5, query_base + query_id] for r in self.ndcg_scorer.relevances]

    return relevances[self.mode]


 def top_querys(self, query_id,querys_a_ordenar):   

    if self.retrieval == 't2i':
        value_retrieval= "i2t" #value=img pasa a ser query para retrieval de text
    if self.retrieval == 'i2t':
        value_retrieval= "t2i"
        
    if value_retrieval == 't2i': #retrieval img quer
                query_base = self.npts * 5 * self.fold_index #0
                relevances = [r[query_base + query_id, self.fold_index * self.npts : (self.fold_index + 1) * self.npts] for r in self.ndcg_scorer.relevances]
    if value_retrieval == 'i2t':
                query_base = self.npts * self.fold_index #0
                relevances = [r[self.fold_index * self.npts * 5 : (self.fold_index + 1) * self.npts * 5, query_base + query_id] for r in self.ndcg_scorer.relevances]

                
    return sorted(querys_a_ordenar, key=lambda x: list(relevances[self.mode])[x], reverse= True)[:self.top] 





 def ideal_value(self):
   # normalizarion with top most relevant per query 
  ndcg=0
  for query in self.all_rankings: # each ranking
   ranking= np.argsort(-self.relevance(query))[:self.top] #top by relevance
   metric=0
   i=1

   for value in ranking: #each value
        if i==1: #first 
            sum_numerator=self.relevance(query)[value]
            rel_ac= self.relevance(query)[value]
        else:
            sum_numerator+=self.relevance(query)[value]  * ( (1 - self.alpha)**rel_ac )
            rel_ac+= self.relevance(query)[value]

        metric+= sum_numerator/np.log2(i+1)
        i+=1
      
   ndcg+=metric

  return ndcg/ len(self.all_rankings) #mean



 def calculate(self):
  ndcg=0
  normalization= self.ideal_value()
    
  for query in self.all_rankings: # each ranking
    ranking= self.all_rankings[query][:self.top]
    metric=0
    i=1
    for value in ranking: #each value
     if i==1:
        sum_numerator=self.relevance(query)[value]
        rel_ac= self.relevance(query)[value]
     else:
        sum_numerator+=self.relevance(query)[value]  * ( (1 - self.alpha)**rel_ac )
        rel_ac+= self.relevance(query)[value]
         
     metric+= sum_numerator/np.log2(i+1)
     i+=1
      
    ndcg+=metric/ normalization

  return ndcg/ len(self.all_rankings) #mean


     

## Model

https://keras.io/examples/vision/siamese_network/

In [13]:
import tensorflow as tf
import gc
print(tf.__version__)


2.12.0


In [59]:
tf.math.l2_normalize(np.array([[[1.,0.],[1.,0.] ],[[1.,0.],[1.,0.] ] ]))

<tf.Tensor: shape=(2, 2, 2), dtype=float64, numpy=
array([[[0.5, 0. ],
        [0.5, 0. ]],

       [[0.5, 0. ],
        [0.5, 0. ]]])>

In [55]:
tf.keras.layers.UnitNormalization()(np.array([[[1.,0.],[1.,0.] ],[[1.,0.],[1.,0.] ] ]))

<tf.Tensor: shape=(2, 2, 2), dtype=float32, numpy=
array([[[1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.]]], dtype=float32)>

In [51]:
tf.keras.layers.UnitNormalization()(np.array([[1.,0.],[1.,0.] ]))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.]], dtype=float32)>

In [14]:

class Net(object):
        def __init__(self, visual_matrix, textual_matrix,factor, margen, margen2, act,mode,estimator,phase ,pretrain=False):
            self.factor= factor
            self.batch_size =  512 #VSE++
            self.epochs = 30 
            self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
            #self.patience= 3
            #self.freq_val=5
            self.estimator=estimator
            self.phase=phase
            self.DIS_MODEL_FILE= path+"/"+self.estimator+"/"+mode+"/"+self.phase+str(margen)+"_"+str(margen2)+"_"+str(factor)+"_"+str(act)+"_"
            print(self.DIS_MODEL_FILE)
            self.mode= mode
            self.alpha=margen
            self.alpha_2= margen2
            
            #arquitecture
            if self.estimator=="net":

                layer_name = 'Visual_Enconder'
                neurons = [visual_matrix.shape[1], int((visual_matrix.shape[1] + self.factor)/2), self.factor]
                v_input = tf.keras.Input(shape=(visual_matrix.shape[1],))

                v_output = tf.keras.layers.Dense(units=neurons[0], activation=act,  kernel_regularizer='l2' )(v_input)
                v_output= tf.keras.layers.Dropout(rate= .2, seed= 0)(v_output)
                v_output = tf.keras.layers.Dense(units=neurons[1], activation=act,  kernel_regularizer='l2' )(v_output)
                v_output = tf.keras.layers.Dense(units=neurons[2], activation=act,  kernel_regularizer='l2')(v_output)
                v_output= tf.math.l2_normalize(v_output, axis=1) #tf.keras.layers.UnitNormalization()(v_output)

                self.visual_encoder = tf.keras.Model(inputs=[v_input], outputs=v_output)


                layer_name = 'Textual_Enconder'
                neurons = [textual_matrix.shape[1],int((textual_matrix.shape[1] + self.factor)/2), self.factor]
                t_input = tf.keras.Input(shape=(textual_matrix.shape[1],))
                t_output = tf.keras.layers.Dense(units=neurons[0], activation=act,  kernel_regularizer='l2')(t_input)
                t_output= tf.keras.layers.Dropout(rate= .2, seed= 0)(t_output)
                t_output = tf.keras.layers.Dense(units=neurons[1], activation=act,  kernel_regularizer='l2')(t_output)
                t_output = tf.keras.layers.Dense(units=neurons[2], activation=act,  kernel_regularizer='l2')(t_output)
                t_output=tf.keras.layers.UnitNormalization()(t_output)

                self.text_encoder= tf.math.l2_normalize(t_output) #tf.keras.Model(inputs=[t_input], outputs=t_output)
                
                
                
            if self.estimator=="Linear":

                layer_name = 'Visual_Enconder'
    
                v_input = tf.keras.Input(shape=(visual_matrix.shape[1],))

                v_output = tf.keras.layers.Dense(units=self.factor, use_bias=False )(v_input)
                v_output= tf.keras.layers.UnitNormalization()(v_output) #tf.math.l2_normalize(v_output)  #

                self.visual_encoder = tf.keras.Model(inputs=[v_input], outputs=v_output)


                layer_name = 'Textual_Enconder'
                t_input = tf.keras.Input(shape=(textual_matrix.shape[1],))
                t_output = tf.keras.layers.Dense(units=self.factor,use_bias=False )(t_input)
                t_output= tf.keras.layers.UnitNormalization()(t_output) #tf.math.l2_normalize(t_output)  #

                self.text_encoder= tf.keras.Model(inputs=[t_input], outputs=t_output)


            if pretrain: 

                self.visual_encoder= tf.keras.models.load_model(self.DIS_MODEL_FILE+'visual_encoder_tf')
                self.text_encoder= tf.keras.models.load_model(self.DIS_MODEL_FILE+'text_encoder_tf')
                print("pre train")
            
            
            i_input = tf.keras.Input(shape=(visual_matrix.shape[1],), name="i")
            i_n_input = tf.keras.Input(shape=(visual_matrix.shape[1],), name="i_n")
            c_input = tf.keras.Input(shape=(textual_matrix.shape[1],), name="c")
            c_n_input = tf.keras.Input(shape=(textual_matrix.shape[1],), name="c_n")

             
            i_emb = self.visual_encoder(i_input) 
            i_n_emb = self.visual_encoder(i_n_input)
            c_emb = self.text_encoder(c_input)
            c_n_emb = self.text_encoder(c_n_input)
            
            
            
            i_c= tf.keras.layers.Dot(axes=1)([i_emb, c_emb]) #self.similarity([i_emb,c_emb])
            i_c_n= tf.keras.layers.Dot(axes=1)([i_emb, c_n_emb]) #self.similarity([i_emb,c_n_emb])
            i_n_c= tf.keras.layers.Dot(axes=1)([i_n_emb, c_emb]) #self.similarity([i_n_emb,c_emb])
            
            
            
            i_n_i= tf.keras.layers.Dot(axes=1)([i_n_emb, i_emb]) #self.similarity([i_n_emb,i_emb])
            c_n_c=tf.keras.layers.Dot(axes=1)([c_n_emb, c_emb]) #self.similarity([c_n_emb,c_emb])
            i_n_c_n= tf.keras.layers.Dot(axes=1)([i_n_emb, c_n_emb]) #self.similarity([i_n_emb,c_n_emb])
            
            sim= ( i_c,i_c_n,i_n_c, i_n_i, c_n_c,i_n_c_n)
            
            


            self.model =  tf.keras.Model(inputs=[i_input, i_n_input, c_input,c_n_input], outputs=sim)
            

            self.visual_encoder.compile(optimizer=self.optimizer)
            self.text_encoder.compile(optimizer=self.optimizer)
            self.model.compile(optimizer=self.optimizer)

        

        def loss(self,input_img, img_negative, input_text,text_negative,mask_s_p):

          i_input = tf.nn.embedding_lookup(visual_matrix, input_img) #[_,1280]
          i_n_input = tf.nn.embedding_lookup(visual_matrix, img_negative) #[_,1280]
          c_input = tf.nn.embedding_lookup(textual_matrix, input_text) #[_,1280]
          c_n_input = tf.nn.embedding_lookup(textual_matrix, text_negative) #[_,1280]

          (i_c,i_c_n,i_n_c,i_n_i ,c_n_c,i_n_c_n) = self.model([i_input, i_n_input, c_input,c_n_input])
          
          elements=np.array([[0,0],[0,0]])
          errors=np.array([[0,0],[0,0]])
          if self.phase=="test": 
              elements= (i_c, i_n_c, i_c_n, i_n_i , c_n_c, i_n_c_n)   
              e_1=np.array((elements[0]>elements[3]+self.alpha_2)).sum() #i_c>i_i_n
              e_2=np.array((elements[0]>elements[4]+self.alpha_2)).sum() #i_c>c_c_n
              e_3=np.array((elements[0]>elements[5]+self.alpha_2)).sum() #i_c>i_n_c_n
              errors= (e_1,e_2,e_3)
            
          if self.mode=="F_HN": #full hard negative  
              i2t_t2i_loss= tf.reduce_mean(tf.maximum(self.alpha - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(self.alpha - i_c+i_n_c, 0) )
              v_c_loss=tf.reduce_mean(tf.maximum(self.alpha_2 - i_c+i_n_i, 0) )
              t_c_loss= tf.reduce_mean(tf.maximum(self.alpha_2 - i_c+c_n_c, 0) )
              s_p_loss= tf.reduce_mean(tf.multiply(mask_s_p,tf.maximum(self.alpha_2 - i_c+i_n_c_n, 0)))

              losses= i2t_t2i_loss + v_c_loss + t_c_loss+ s_p_loss
                
          if self.mode=="HN" or self.mode=="random": # hard negative
              i2t_t2i_loss= tf.reduce_mean(tf.maximum(self.alpha - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(self.alpha - i_c+i_n_c, 0) )
              losses= i2t_t2i_loss 
              
            
          if self.mode=="M_HN": # margen hard negative
              i2t_t2i_loss= tf.reduce_mean(tf.maximum(i_n_i - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(c_n_c - i_c+i_n_c, 0) )
              #i2t_t2i_loss= tf.reduce_mean(tf.maximum(c_n_c - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(i_n_i - i_c+i_n_c, 0) )

              losses= i2t_t2i_loss 
          
          return losses,elements,errors
        

        def predict(self,query, retrieval,split):
            if type(query) is not list:
                query= [query]
            if retrieval=="t2i":
                c_input = self.text_encoder(tf.nn.embedding_lookup(textual_matrix, query))
                if split=="val":
                    i_eval= self.visual_encoder(visual_matrix[ self.val_indices_])
                if split=="test":
                    i_eval= self.visual_encoder(visual_matrix[self.test_indices_])
                if split=="train":
                    i_eval=self.visual_encoder(visual_matrix[self.train_indices_]) 
                
                rating= tf.matmul(i_eval, tf.transpose(c_input))
            
            if retrieval=="i2t": 
                i_input = self.visual_encoder(tf.nn.embedding_lookup(visual_matrix, query))
                if split=="val":
                    c_eval= self.text_encoder(textual_matrix[self.data_text_Val])
                if split=="test":
                    c_eval= self.text_encoder(textual_matrix[self.data_text_Test])
                if split=="train":
                    c_eval=self.text_encoder(textual_matrix[self.data_text_])                
        
                rating= tf.matmul(i_input, tf.transpose(c_eval) )

            if retrieval=="i2i":
                if split=="test":
                    i_query = self.visual_encoder(tf.nn.embedding_lookup(visual_matrix, query))
                    i_eval=  self.visual_encoder(visual_matrix[self.test_indices_]) 
                    rating= tf.matmul(i_query, tf.transpose(i_eval) )
            if retrieval=="t2t":
                if split=="test":
                    c_query = self.text_encoder(tf.nn.embedding_lookup(textual_matrix, query))
                    c_eval= self.text_encoder(textual_matrix[self.data_text_Test])
                    rating= tf.matmul(c_eval , tf.transpose(c_query)  )

            return np.reshape(rating, [-1]) 


        def inference(self):
            
          data = []
          self.data_text_=[]

          self.val_indices_=  [ universe_img.index(i) for i in val_r_text.keys() ]
          self.train_indices_= [universe_img.index(i) for i in train_indices]
          self.data_text_Val=  [ universe_text.index(i) for i in val_r_img.keys() ]

          for i in self.train_indices_:
              for t in all_r_text[ train_indices[self.train_indices_.index(i)] ]:
                  data.append([ universe_text.index(t), i ]) # [texto, img]
                  self.data_text_.append(universe_text.index(t))
    
          if self.phase=="test":
              self.test_indices_= [universe_img.index(i) for i in test_r_text.keys()]
              self.data_text_Test= [universe_text.index(i) for i in test_r_img.keys()]
          
          print("inference")
          results= evaluate_model(self, nDCG=True, split="test", retrieval="t2i")
          print("1->2")
          result_t = evaluate_model(self, nDCG=False, split="test", retrieval="i2t")
          np.save(self.DIS_MODEL_FILE+"test_metrics.npy", np.array([results,result_t], dtype=object) )
                
                

        def save(self, model_time, loss_c,recalls,epoch, best_recall ):
          gpu_name=!nvidia-smi -L
          with open(self.DIS_MODEL_FILE+'train.txt', 'w') as f:
            f.write("training time of model: %fs" % (model_time) + "\n")
            f.write("finish epoch %s" % epoch + "with recall %s" % best_recall  + "\n")
            f.write(gpu_name[0] + "\n")
          np.save(self.DIS_MODEL_FILE+"Loss_recalls.npy", np.array([loss_c,recalls],dtype=object))
          
          self.visual_encoder.save(self.DIS_MODEL_FILE+'visual_encoder_tf',save_format='tf')
          self.text_encoder.save(self.DIS_MODEL_FILE+'text_encoder_tf',save_format='tf')


        def training(self):
            
            data = []
            self.data_text_=[]

            self.val_indices_=  [ universe_img.index(i) for i in val_r_text.keys() ]
            self.train_indices_= [universe_img.index(i) for i in train_indices]
            self.data_text_Val=  [ universe_text.index(i) for i in val_r_img.keys() ]

            for i in self.train_indices_:
                for t in all_r_text[ train_indices[self.train_indices_.index(i)] ]:
                    data.append([ universe_text.index(t), i ]) # [texto, img]
                    self.data_text_.append(universe_text.index(t))
    
            if self.phase=="test":
                self.test_indices_= [universe_img.index(i) for i in test_r_text.keys()]
                self.data_text_Test= [universe_text.index(i) for i in test_r_img.keys()]
                
            recalls=[]
            loss_c=[]
            best_recall= 0
            start_time = time.time() 

            train_size = len(data)
            c_images= len(self.train_indices_)
            
                

            for epoch in range(self.epochs):
                
        
                if epoch>=15:
                    self.optimizer.learning_rate.assign(0.00002)
                    
                print("epoch: %d" % epoch)
                np.random.shuffle(data)
                index = 0
                loss_c_=[]
                loss_elements=[]
                loss_errores=[]
                

                while index + self.batch_size < train_size:
                    #print((index + self.batch_size)/train_size)
                    input_text, input_img = get_batch_data(data, index,self.batch_size)  
                    index += self.batch_size
                    mask_s_p= [0]*self.batch_size
                    
                    
                    if self.mode=="random": #train quitando - minibatch (positivos)
                      text_negative= np.random.choice( list( set( self.data_text_ )- set(input_text) ) , self.batch_size, replace=False)
                      img_negative= np.random.choice(list( set(self.train_indices_) - set(input_img) ), self.batch_size, replace=False)
                    
                    else: 
                        img_negative,text_negative=[],[] #negatives
                        
                        rating= np.array( self.predict( list(input_text) ,"t2i", "train"), dtype=float)
                        gc.collect() 
                        rating_text= np.array(self.predict( list(input_img) ,"i2t", "train"), dtype=float)
                        gc.collect()
                        
                        for i_ in range(self.batch_size):
                            ratings_batch= rating[c_images*i_:c_images*(i_+1)] # t2i query minibatch
                            ratings_text_batch= rating_text[c_images*5*i_:c_images*5*(i_+1)]  # i2t query minibatch
                    
                            ratings_batch[train_indices.index(universe_img[input_img[i_]])]= - np.infty
                            five_text= all_r_text[all_r_img[ universe_text[input_text[i_]]] ]
                            ratings_text_batch[ [ data_text.index(j) for j in five_text ] ]= - np.infty

                            img_negative.append(self.train_indices_[np.argmax(ratings_batch)])
                            text_negative.append(self.data_text_[np.argmax(ratings_text_batch)])
                                             #sp condition
                            if  self.mode=="F_HN" and all_r_img[ universe_text[text_negative[-1] ]]  == universe_img[img_negative[-1]] :
                                mask_s_p[i_]=1
                            del ratings_batch
                            del ratings_text_batch
                    
                    with tf.GradientTape() as tape:
                            losses, elements, errors= self.loss(input_img, img_negative, input_text,text_negative, mask_s_p)
                            loss_elements.append( [np.mean(i,axis=0) for i in elements ] )
                            loss_errores.append(errors)
                            
                    grads = tape.gradient(losses, self.model.trainable_variables)
                    self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
                    
                    
                print(time.time() - start_time)

                
                loss_c.append(( losses.numpy(), np.mean(loss_elements,axis=0),  np.sum(loss_errores,axis=0)))
                
                """
                if self.phase=="test" and epoch%self.freq_val ==0:
                    print("validation")
                    r_t2i= evaluate_model(self,nDCG=False, split="val", retrieval="t2i")[0][0]
                    r_i2t= evaluate_model(self,nDCG=False, split="val", retrieval="i2t")[0][0]

                    r_sum=np.sum(r_t2i)  +  np.sum(r_i2t) 


                    print( "Recall t2i-i2t: ", list(r_t2i)+list(r_i2t))
                    recalls.append((r_t2i,r_i2t, r_sum))

                    if r_sum-np.sum(best_recall)>0.1:
                        best_recall=list(r_t2i)+list(r_i2t)
                        wait_for = 0
                        self.save( start_time , loss_c,recalls,epoch,best_recall)

                    else:
                        wait_for += 1
                        if wait_for == self.patience:
                          print("break train")
                          break
                """
                    
             
            model_time= time.time() - start_time
            print(model_time)
            """
            if self.phase=="test":
                self.save( model_time , loss_c,recalls,epoch,best_recall)
                print("evaluating")
                self.inference()
                
            if self.phase=="val":
                print("validation")
                r_t2i= evaluate_model(self,nDCG=False, split="val", retrieval="t2i")[0][0]
                r_i2t= evaluate_model(self,nDCG=False, split="val", retrieval="i2t")[0][0]

                best_recall=list(r_t2i)+list(r_i2t)
                #self.save( model_time , loss_c,recalls,epoch,best_recall)
                print( "Recall t2i-i2t: ", best_recall) 
            """

In [15]:
net =  Net(visual_matrix,textual_matrix,1024,0.2,0.2, None,"M_HN","Linear","test", pretrain=True) 
net.inference()

COCO/Linear/M_HN/test0.2_0.2_1024_None_
pre train
inference
1->2


In [14]:
net =  Net(visual_matrix,textual_matrix,1024,0.2,0.2, None,"HN","Linear","val") 
net.training()

epoch: 0
30.702526807785034
epoch: 1
60.461031436920166
epoch: 2
90.40238094329834
epoch: 3
120.12043118476868
epoch: 4
150.25114226341248
150.2562427520752


l2 norm:
Recall t2i-i2t:  [35.18518518518518, 74.07407407407408, 92.5925925925926, 61.111111111111114, 87.03703703703704, 94.44444444444444]

Recall t2i-i2t:  [58.51851851851852, 89.25925925925925, 96.66666666666667, 77.77777777777777, 96.29629629629629, 98.14814814814815]

unit norm:
Recall t2i-i2t:  [58.888888888888886, 89.25925925925925, 95.92592592592592, 79.62962962962963, 96.29629629629629, 98.14814814814815]

Recall t2i-i2t:  [54.44444444444444, 89.25925925925925, 95.18518518518519, 74.07407407407408, 96.29629629629629, 100.0]


In [25]:
net =  Net(visual_matrix,textual_matrix,1024,0.4,0, None,"HN","Linear","test") 
net.training()

epoch: 0


ResourceExhaustedError: Exception encountered when calling layer "dense_1" "                 f"(type Dense).

{{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[566435,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:MatMul]

Call arguments received by layer "dense_1" "                 f"(type Dense):
  • inputs=tf.Tensor(shape=(566435, 768), dtype=float32)

In [37]:
for m in m_validation:
    print(m)
    net =  Net(visual_matrix,textual_matrix,1024,m,0, None,"HN","Linear","test") 
    net.training()

0.2
epoch: 0


ResourceExhaustedError: Exception encountered when calling layer "unit_normalization_3" "                 f"(type UnitNormalization).

{{function_node __wrapped__Square_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Square]

Call arguments received by layer "unit_normalization_3" "                 f"(type UnitNormalization):
  • inputs=tf.Tensor(shape=(283215, 1024), dtype=float32)

In [18]:
for f in f_validation:
    for m in m_validation:
        print(m,f)
        net =  Net(visual_matrix,textual_matrix,f,m,0, None,"HN","Linear","val") 
        net.training()

0.2 896
epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
INFO:tensorflow:Assets written to: F30K/Linear/HN/val_0.2_0_896_None_visual_encoder_tf\assets
INFO:tensorflow:Assets written to: F30K/Linear/HN/val_0.2_0_896_None_text_encoder_tf\assets
Recall t2i-i2t:  [26.232741617357004, 56.68639053254438, 69.09270216962524, 44.47731755424063, 71.99211045364892, 82.54437869822485]
0.4 896
epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
INFO:tensorflow:Assets written to: F30K/Linear/HN/val_0.4_0_896

KeyboardInterrupt: 

In [22]:
for margen in [0.2, 0.6]  :
    print(margen)
    net =  Net(visual_matrix,textual_matrix,1024,0.4,margen, None,"F_HN","Linear","val") 
    net.training()

0.2
epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
INFO:tensorflow:Assets written to: F30K/Linear/F_HN/val_0.4_0.2_1024_None_visual_encoder_tf\assets
INFO:tensorflow:Assets written to: F30K/Linear/F_HN/val_0.4_0.2_1024_None_text_encoder_tf\assets
Recall t2i-i2t:  [35.285996055226825, 64.93096646942801, 75.70019723865877, 40.92702169625247, 74.16173570019724, 84.91124260355029]
0.6
epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
INFO:tensorflow:Assets written to: F30K/Linear/F_HN/val_0.4_0

In [16]:
#net =  Net(visual_matrix,textual_matrix,1024,0.4,0.4, None,"HN","Linear","test") 
#net.training()

#net =  Net(visual_matrix,textual_matrix,1024,0.4,0.4, None,"F_HN","Linear","test") 
#net.training()

net =  Net(visual_matrix,textual_matrix,1024,0.4,0.4, None,"M_HN","Linear","test") 
net.training()

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
INFO:tensorflow:Assets written to: F30K/Linear/M_HN/test_otro_0.4_0.4_1024_None_visual_encoder_tf\assets
INFO:tensorflow:Assets written to: F30K/Linear/M_HN/test_otro_0.4_0.4_1024_None_text_encoder_tf\assets
evaluating


In [15]:
for act in ["relu","tanh"]  :
    print(act)
    net =  Net(visual_matrix,textual_matrix,712,0.4,0, act,"HN","net","val") 
    net.training()

relu
epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
INFO:tensorflow:Assets written to: F30K/net/HN/val_0.4_0_712_relu_visual_encoder_tf\assets
INFO:tensorflow:Assets written to: F30K/net/HN/val_0.4_0_712_relu_text_encoder_tf\assets
Recall t2i-i2t:  [16.429980276134124, 39.447731755424066, 52.14990138067061, 4.812623274161735, 19.171597633136095, 29.447731755424062]
tanh
epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
INFO:tensorflow:Assets written to: F30K/net/HN/val_0.4_0_712_tanh_visual

In [47]:
net =  Net(visual_matrix,textual_matrix,1024,0.4,0, "tanh","HN","net","val") 
net.training()

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
INFO:tensorflow:Assets written to: F30K/net/HN/val_0.4_0_1024_tanh_visual_encoder_tf\assets
INFO:tensorflow:Assets written to: F30K/net/HN/val_0.4_0_1024_tanh_text_encoder_tf\assets
Recall t2i-i2t:  [25.502958579881657, 54.891518737672584, 67.57396449704142, 8.422090729783038, 31.26232741617357, 45.26627218934911]


# Examples

In [12]:
test_indices_= [universe_img.index(i) for i in test_r_text.keys()]
data_text_Test= [universe_text.index(i) for i in test_r_img.keys()]
all_values= all_test_indices_img 
all_indices= all_test_indices_text 
eval_values= list(test_r_text.keys())
eval_indices= list(test_r_img.keys())

## F HN i2i

In [13]:
DIS_MODEL_FILE= path+"/Linear/F_HN/test_0.4_0.4_1024_None_"

visual_encoder= tf.keras.models.load_model(DIS_MODEL_FILE+'visual_encoder_tf')
text_encoder= tf.keras.models.load_model(DIS_MODEL_FILE+'text_encoder_tf')

232 + 1 el mejor
564 + 4 el peor R.
946*5+3 el peor S

In [56]:

#if retrieval=="i2i":
#if split=="test":
index= 232
i= 1 
print("Query img id", eval_values[index])
query=[ universe_img.index(eval_values[index])  ]

Query img id 7739


In [58]:

i_query = visual_encoder(tf.nn.embedding_lookup(visual_matrix, query))
i_eval=  visual_encoder(visual_matrix[test_indices_]) 
rating= tf.matmul(i_query, tf.transpose(i_eval) )

indices_= np.argsort( -np.array(rating) ) 
inds= [all_values.index(eval_values[j]) for j in indices_[0]] 


sorted_indexes = inds[:5] 
print("top 5: imgid", [all_values[j] for j in sorted_indexes] )

ndcg_scorer = DCG( rank=25, relevance_methods=['rougeL', 'spice'],split="test")
rougel_ndcgs = np.zeros(5000)
spice_ndcgs = np.zeros(5000)
for z in range(5):
    #print(all_indices[all_indices.index(eval_indices[5 * index + i])])
    rougel_ndcgs[5 * index + z], spice_ndcgs[5 * index + z] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[5 * index + z]), inds, retrieval='t2i').values()
print("R: ",  np.round(np.mean(rougel_ndcgs[rougel_ndcgs != 0]),3) )
print("S: ", np.round(np.mean(spice_ndcgs[spice_ndcgs != 0]),3) )

top 5: imgid [7739, 10577, 8766, 13516, 13990]
R:  0.804
S:  0.777


In [22]:
for im in [all_values[j] for j in sorted_indexes]:
    print(im)
    for c in range(5):
        print(dataset[im]["sentences"][c]["raw"])

7739
A dog jumping over high jump poles at show.
A black and white dog jumps over a hurdle.
A small dog jumps over a striped gate.
A dog jumps over an obstacle outside.
A small dog leaps a barrier.
10577
A black and white dog jumping over a steeple vault at a competition
A black and white dog performs a jump with his owner looking on.
A black and white dog running through an obstacle course
A black and white dog is jumping fences in a dog show.
A dog leaping over a hurdle.
8766
A dog is jumping over a series of colored gates.
A black and white dog is jumping over a hurdle.
A dog hurdles over a series of poles.
A small black dog jumping over gates
A dog jumping over a beam
13516
A black and white dog is playing with an orange ball in the snow.
The little black and white dog is in the snow with a red ball.
A black-and-white dog goes for a flattened ball on the snow.
A black dog is chasing a punctured ball in the snow.
A dog getting a ball in the snow
13990
The brown dog is jumping a hurd

In [16]:

relevance_filenames= [path+"/f30k-test-rougeL.npy", path+"/f30k-test-spice.npy"] #500.000=5000x1000
                      
relevances = [np.memmap(f, dtype=np.float32, mode='r') for f in relevance_filenames]
        
        # una fila se lleva a una matriz de n_queries filas y lo que quede de columnas (en orden)
for r in relevances:
    r.shape = (1000*5, -1) #debe dividir a 5 millones

#if retrieval == 't2i':
            #caption query pertenece a  [0,5000] valores de relevancia en vector [0,1000] imagenes
            # (5000,1000)
#R= np.zeros(5)
#S=np.zeros(5)
#for i in range(5):
query_id= all_indices.index(eval_indices[5 * index + i])
relevances_ = [r[ query_id, 0 : 1000] for r in relevances]
R=relevances_[0][sorted_indexes]
S= relevances_[1][sorted_indexes]
print("R:", np.round(R,3) )
print("S: ", np.round(S,3) )

R: [1.    0.81  0.834 0.433 0.639]
S:  [0.417 0.385 0.5   0.273 0.273]


#  HN t2i

In [23]:
DIS_MODEL_FILE= path+"/Linear/HN/test_0.4_0.4_1024_None_"

visual_encoder_= tf.keras.models.load_model(DIS_MODEL_FILE+'visual_encoder_tf')
text_encoder_= tf.keras.models.load_model(DIS_MODEL_FILE+'text_encoder_tf')

In [34]:
dataset[ eval_values[index]]["imgid"]

7739

In [72]:
i=4
eval_indices[5 * index + i]

38699

In [63]:
dataset[ eval_values[index]]["sentences"][eval_indices[5 * index + i]%5]["raw"]

'A black and white dog jumps over a hurdle.'

In [73]:

query=[  universe_text.index(eval_indices[5 * index + i])  ]
print("Query text id", eval_indices[5 * index + i])


c_input = text_encoder_(tf.nn.embedding_lookup(textual_matrix, query))
i_eval= visual_encoder_(visual_matrix[test_indices_])
rating= tf.matmul(i_eval, tf.transpose(c_input))
rating= np.reshape(rating, [-1]) 

indices_= np.argsort( -np.array(rating) ) 
inds= [all_values.index(eval_values[j]) for j in indices_] 


sorted_indexes = inds[:5] 
print("top 5: imgid", [all_values[j] for j in sorted_indexes] )

ndcg_scorer = DCG( rank=25, relevance_methods=['rougeL', 'spice'],split="test")
rougel_ndcgs = np.zeros(5000)
spice_ndcgs = np.zeros(5000)

rougel_ndcgs[5 * index + i], spice_ndcgs[5 * index + i] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[5 * index + i]), inds, retrieval='t2i').values()
print("R: ",  np.round(np.mean(rougel_ndcgs[rougel_ndcgs != 0]),3) ) 
print("S: ", np.round(np.mean(spice_ndcgs[spice_ndcgs != 0]),3) )

Query text id 38699
top 5: imgid [13785, 13135, 12356, 13516, 8766]
R:  0.635
S:  0.616


In [74]:

#relevance_filenames= [path+"/f30k-test-rougeL.npy", path+"/f30k-test-spice.npy"] #500.000=5000x1000
                      
#relevances = [np.memmap(f, dtype=np.float32, mode='r') for f in relevance_filenames]
        
        # una fila se lleva a una matriz de n_queries filas y lo que quede de columnas (en orden)
#for r in relevances:
    #r.shape = (1000*5, -1) #debe dividir a 5 millones

#if retrieval == 't2i':
            #caption query pertenece a  [0,5000] valores de relevancia en vector [0,1000] imagenes
            # (5000,1000)


query_id= all_indices.index(eval_indices[5 * index + i])
relevances_ = [r[ query_id, 0 : 1000] for r in relevances]
R=relevances_[0][sorted_indexes]
S= relevances_[1][sorted_indexes]
print("R:", np.round(R,3))
print("S: ", np.round(S,3))

R: [0.359 0.347 0.12  0.418 0.5  ]
S:  [0.074 0.091 0.083 0.095 0.211]


In [146]:
sum_R=[]
sum_S=[]
indices=[]

for index in tqdm(range(len(eval_values))):
    for i in range(5):
        
        query=[ universe_img.index(eval_values[index])  ]

        i_query = visual_encoder(tf.nn.embedding_lookup(visual_matrix, query))
        i_eval=  visual_encoder(visual_matrix[test_indices_]) 
        rating= tf.matmul(i_query, tf.transpose(i_eval) )

        indices_= np.argsort( -np.array(rating) ) 
        inds= [all_values.index(eval_values[j]) for j in indices_[0]] 


        sorted_indexes = inds[:5] 
        
        query_id= all_indices.index(eval_indices[5 * index + i])
        relevances_ = [r[ query_id, 0 : 1000] for r in relevances]
        R_FHN=np.sum(relevances_[0][sorted_indexes])
        S_FHN= np.sum(relevances_[1][sorted_indexes])
        
        #############################################
        
        query=[  universe_text.index(eval_indices[5 * index + i])  ]


        c_input = text_encoder_(tf.nn.embedding_lookup(textual_matrix, query))
        i_eval= visual_encoder_(visual_matrix[test_indices_])
        rating= tf.matmul(i_eval, tf.transpose(c_input))
        rating= np.reshape(rating, [-1]) 
        indices_= np.argsort( -np.array(rating) ) 
        inds= [all_values.index(eval_values[j]) for j in indices_] 
        sorted_indexes = inds[:5] 
        query_id= all_indices.index(eval_indices[5 * index + i])
        relevances_ = [r[ query_id, 0 : 1000] for r in relevances]
        R=np.sum(relevances_[0][sorted_indexes])
        S= np.sum(relevances_[1][sorted_indexes])
        
        sum_R.append( R_FHN - R)
        sum_S.append(  S_FHN - S)
        indices.append(5 * index + i)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [19:33<00:00,  1.17s/it]


In [148]:
np.argsort(-np.array(sum_R))

array([1163, 1161, 4674, ...,   79, 2508, 2824], dtype=int64)

In [149]:
np.argsort(-np.array(sum_S))

array([1161, 2480,  775, ...,  474,  399, 4733], dtype=int64)

In [171]:
4733//5

946

In [173]:
946*5+3

4733